In [1]:
import pandas as pd
import glob
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import seaborn as sns

In [2]:
# 파이썬 구글 빅쿼리 연동 코드
# json 파일

key_path = glob.glob("*.json")[0]
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)

In [3]:
# sql 추출 및 데이터 프레임 변환

def sql_to_dataframe(sql:str) -> pd.DataFrame:
    """
    Args:
        sql (str): sql for extraction

    Returns:
        pd.DataFrame: extract data with sql
    """
    query_job = client.query(sql)
    df = query_job.to_dataframe()
    return df

### Recency 얼마나 최근 구매했는가?

In [4]:
# 데이터 불러오기

sql = """
SELECT
  Customer_ID,
  max_order_date,
  MAX(max_order_date) OVER(ORDER BY max_order_date DESC) AS year_max_order_date,
  DATE_DIFF(MAX(max_order_date) OVER(ORDER BY max_order_date DESC),max_order_date,day) AS recency
FROM(
  SELECT
    Customer_ID,
    MAX(Order_Date) AS max_order_date,
  FROM `rfm-analysis-392707.rfm_analysis.rfm_analysis`
  GROUP BY Customer_ID
)
"""
df = sql_to_dataframe(sql)
df

,Customer_ID,max_order_date,year_max_order_date,recency
0,CC-12430,2020-12-30,2020-12-30,0
1,PO-18865,2020-12-30,2020-12-30,0
2,JM-15580,2020-12-30,2020-12-30,0
3,EB-13975,2020-12-30,2020-12-30,0
4,JG-15160,2020-12-29,2020-12-30,1
...,...,...,...,...
688,TP-21565,2020-01-14,2020-12-30,351
689,DA-13450,2020-01-12,2020-12-30,353
690,TS-21085,2020-01-07,2020-12-30,358
691,DP-13390,2020-01-06,2020-12-30,359


### Frequency 얼마나 자주 구매했는가?

In [5]:
sql = """
SELECT
  Customer_ID,
  COUNT(DISTINCT Order_ID) AS frequency
FROM `rfm-analysis-392707.rfm_analysis.rfm_analysis`
GROUP BY Customer_ID
"""
df = sql_to_dataframe(sql)
df

,Customer_ID,frequency
0,NW-18400,1
1,TM-21010,3
2,ML-17755,6
3,RS-19870,1
4,PO-18850,5
...,...,...
688,MO-17950,1
689,JF-15565,1
690,KS-16300,1
691,EB-14170,1


### monetary 얼마나 많은 금액을 지출했는가?

In [6]:
sql = """
SELECT
  Customer_ID,
  SUM(Sales) AS monetary
FROM `rfm-analysis-392707.rfm_analysis.rfm_analysis`
GROUP BY Customer_ID
"""
df = sql_to_dataframe(sql)
df

,Customer_ID,monetary
0,NW-18400,30.320
1,TM-21010,536.336
2,ML-17755,1913.104
3,RS-19870,12.680
4,PO-18850,5308.107
...,...,...
688,MO-17950,24.000
689,JF-15565,831.368
690,KS-16300,61.876
691,EB-14170,10.816


### RFM 지표

In [7]:
sql = """
SELECT
  Customer_ID,
  DATE_DIFF(MAX(max_order_date) OVER(ORDER BY max_order_date DESC),max_order_date,day) AS recency,
  frequency,
  monetary
FROM(
  SELECT
    Customer_ID,
    MAX(Order_Date) AS max_order_date,
    COUNT(DISTINCT Order_ID) AS frequency,
    SUM(Sales) AS monetary
  FROM `rfm-analysis-392707.rfm_analysis.rfm_analysis`
  GROUP BY Customer_ID
)
"""
df = sql_to_dataframe(sql)
df

,Customer_ID,recency,frequency,monetary
0,CC-12430,0,4,1580.552
1,PO-18865,0,4,1139.462
2,JM-15580,0,1,3.024
3,EB-13975,0,1,34.624
4,JG-15160,1,3,1608.310
...,...,...,...,...
688,TP-21565,351,1,337.688
689,DA-13450,353,1,77.760
690,TS-21085,358,1,2.808
691,DP-13390,359,2,37.340


### RFM 구분 비율

In [8]:
sql = """
SELECT
  *,
  PERCENTILE_CONT(recency,0.25) OVER() AS recency_q1,
  PERCENTILE_CONT(recency,0.5) OVER() AS recency_q2,
  PERCENTILE_CONT(recency,0.75) OVER() AS recency_q3,

  PERCENTILE_CONT(frequency,0.25) OVER() AS frequency_q1,
  PERCENTILE_CONT(frequency,0.5) OVER() AS frequency_q2,
  PERCENTILE_CONT(frequency,0.75) OVER() AS frequency_q3,

  PERCENTILE_CONT(monetary,0.25) OVER() AS monetary_q1,
  PERCENTILE_CONT(monetary,0.5) OVER() AS monetary_q2,
  PERCENTILE_CONT(monetary,0.75) OVER() AS monetary_q3,
FROM(
  SELECT
    Customer_ID,
    DATE_DIFF(MAX(max_order_date) OVER(ORDER BY max_order_date DESC),max_order_date,day) AS recency,
    frequency,
    monetary,
  FROM(
    SELECT
      Customer_ID,
      MAX(Order_Date) AS max_order_date,
      COUNT(DISTINCT Order_ID) AS frequency,
      SUM(Sales) AS monetary
    FROM `rfm-analysis-392707.rfm_analysis.rfm_analysis`
    GROUP BY Customer_ID
  )
)
"""

df = sql_to_dataframe(sql)
df

,Customer_ID,recency,frequency,monetary,recency_q1,recency_q2,recency_q3,frequency_q1,frequency_q2,frequency_q3,monetary_q1,monetary_q2,monetary_q3
0,ML-17395,69,2,85.500,28.0,57.0,118.0,1.0,2.0,3.0,204.405,604.91,1334.6416
1,DV-13045,19,3,1081.883,28.0,57.0,118.0,1.0,2.0,3.0,204.405,604.91,1334.6416
2,JP-15460,37,2,266.421,28.0,57.0,118.0,1.0,2.0,3.0,204.405,604.91,1334.6416
3,DK-13375,19,3,520.406,28.0,57.0,118.0,1.0,2.0,3.0,204.405,604.91,1334.6416
4,LW-16825,88,1,20.640,28.0,57.0,118.0,1.0,2.0,3.0,204.405,604.91,1334.6416
...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,TS-21205,198,1,2942.784,28.0,57.0,118.0,1.0,2.0,3.0,204.405,604.91,1334.6416
689,KM-16720,28,1,1271.170,28.0,57.0,118.0,1.0,2.0,3.0,204.405,604.91,1334.6416
690,AC-10615,6,4,2386.926,28.0,57.0,118.0,1.0,2.0,3.0,204.405,604.91,1334.6416
691,DB-13120,48,2,771.540,28.0,57.0,118.0,1.0,2.0,3.0,204.405,604.91,1334.6416


### RFM Score

In [9]:
sql = """
SELECT
  Customer_ID,
  recency,
  frequency,
  monetary,
  CASE
    WHEN recency >= recency_q3 THEN 1
    WHEN recency < recency_q3 AND recency >= recency_q2 THEN 2
    WHEN recency < recency_q2 AND recency >= recency_q1 THEN 3
    WHEN recency < recency_q1 THEN 4
  END recency_score,
  CASE
    WHEN frequency >= frequency_q3 THEN 4
    WHEN frequency < frequency_q3 AND frequency >= frequency_q2 THEN 3
    WHEN frequency < frequency_q2 AND frequency >= frequency_q1 THEN 2
    WHEN frequency < frequency_q1 THEN 1
  END frequency_score,
  CASE
    WHEN monetary >= monetary_q3 THEN 4
    WHEN monetary < monetary_q3 AND monetary >= monetary_q2 THEN 3
    WHEN monetary < monetary_q2 AND monetary >= monetary_q1 THEN 2
    WHEN monetary < monetary_q1 THEN 1
  END monetary_score
FROM(
  SELECT
    *,
    PERCENTILE_CONT(recency,0.25) OVER() AS recency_q1,
    PERCENTILE_CONT(recency,0.5) OVER() AS recency_q2,
    PERCENTILE_CONT(recency,0.75) OVER() AS recency_q3,

    PERCENTILE_CONT(frequency,0.25) OVER() AS frequency_q1,
    PERCENTILE_CONT(frequency,0.5) OVER() AS frequency_q2,
    PERCENTILE_CONT(frequency,0.75) OVER() AS frequency_q3,

    PERCENTILE_CONT(monetary,0.25) OVER() AS monetary_q1,
    PERCENTILE_CONT(monetary,0.5) OVER() AS monetary_q2,
    PERCENTILE_CONT(monetary,0.75) OVER() AS monetary_q3,
  FROM(
    SELECT
      Customer_ID,
      DATE_DIFF(MAX(max_order_date) OVER(ORDER BY max_order_date DESC),max_order_date,day) AS recency,
      frequency,
      monetary,
    FROM(
      SELECT
        Customer_ID,
        MAX(Order_Date) AS max_order_date,
        COUNT(DISTINCT Order_ID) AS frequency,
        SUM(Sales) AS monetary
      FROM `rfm-analysis-392707.rfm_analysis.rfm_analysis`
      GROUP BY Customer_ID
    )
  )
)
"""

df = sql_to_dataframe(sql)
df

,Customer_ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,BE-11455,254,1,979.626,1,2,3
1,SC-20440,36,3,502.688,3,4,2
2,HR-14830,159,2,3124.806,1,3,4
3,CM-11815,131,3,1589.904,1,4,4
4,RB-19330,68,1,337.818,2,2,2
...,...,...,...,...,...,...,...
688,RA-19885,43,3,1511.296,3,4,4
689,EB-13930,161,1,526.344,1,2,2
690,SK-19990,8,2,876.249,4,3,3
691,LL-16840,249,2,121.554,1,3,1
